# Cleaning and exploring Data

In [89]:
import pandas as pd
import matplotlib.pyplot as plt 
from mpl_toolkits import mplot3d

In [90]:
df = pd.read_csv('Data/raw.csv', header=None)

In [91]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,591,592,593,594,595,596,597,598,599,600
0,1.0,0.013694,0.024678,0.130142,0.000040,-0.092436,-0.084620,0.001543,0.019859,0.120984,...,0.127987,-0.000244,0.005662,-0.000609,0.004237,0.010671,0.128107,-0.000149,0.004620,-0.000293
1,1.0,0.000885,0.009893,0.126730,-0.000118,-0.010667,-0.021169,-0.003575,0.011060,0.124755,...,0.129274,0.000009,0.005978,0.003971,0.001723,0.009085,0.127807,0.000072,0.006862,0.000465
2,1.0,0.002860,0.013634,0.127299,0.000135,0.008220,-0.033392,0.007499,0.017285,0.131818,...,0.128017,0.000040,-0.003118,0.003624,0.000316,0.007169,0.127418,0.000009,-0.003845,0.003718
3,1.0,-0.027728,0.012108,0.129483,0.000009,0.004620,-0.096685,-0.021113,0.015160,0.125892,...,0.127628,0.000040,-0.002929,-0.003104,-0.006029,0.011898,0.127418,0.000040,-0.003687,-0.000198
4,1.0,-0.000253,0.012766,0.125084,0.000040,0.007967,-0.028718,0.006811,0.014831,0.124844,...,0.127897,-0.000055,-0.001381,-0.000040,0.001304,0.012886,0.128047,-0.000118,-0.003813,0.000497


In [92]:
num_features = 5
num_labels = 3
df_list = []

In [93]:
start_index = 0
end_index = num_features
for i in range(num_labels):
    df_list.append(df.iloc[start_index:end_index:])
    start_index+=num_features
    end_index+=num_features

In [94]:
df_list[2].head()

,0,1,2,3,4,5,6,7,8,9,...,591,592,593,594,595,596,597,598,599,600
10,2.0,0.019560,0.037158,0.123019,-0.000118,-0.026174,-0.038003,0.014802,0.036979,0.110568,...,0.173659,0.000009,0.044636,0.028859,0.009534,0.004386,0.176861,0.000040,0.039803,0.088899
11,2.0,-0.010818,-0.007107,0.127688,0.000009,0.000956,0.011993,-0.011356,-0.008723,0.130411,...,0.125623,0.000040,0.048394,-0.026539,-0.040477,-0.010998,0.127837,-0.000055,0.035824,-0.023728
12,2.0,0.012826,-0.004623,0.127418,0.000166,0.009673,-0.071987,0.028330,0.011479,0.135649,...,0.166266,-0.000023,-0.067769,0.090920,-0.002228,-0.036138,0.159203,0.000166,-0.087225,0.026237
13,2.0,0.014263,0.010073,0.122241,0.000198,-0.036028,-0.063617,0.017016,0.006152,0.121403,...,0.107605,0.000198,-0.010193,0.091394,-0.016235,-0.026082,0.113950,0.000135,-0.011709,0.079645
14,2.0,0.029108,0.002680,0.124246,0.000072,-0.033312,-0.073566,0.028180,0.003458,0.130142,...,0.113801,-0.000149,-0.026869,-0.018295,-0.023328,-0.034851,0.122719,0.000103,-0.011741,-0.005946


In [95]:
for i in range(len(df_list)):
    df_list[i] = df_list[i].reset_index(drop=True)
    df_list[i] = df_list[i].transpose()
    df_list[i]['Label'] = df_list[i][0][0]
    df_list[i] = df_list[i].iloc[1:]

## Making a new dataframe (assuming that this is the right format)

In [96]:
df_main = pd.concat(df_list)

## Final data to be processed

In [97]:
df_main

,0,1,2,3,4,Label
1,0.013694,0.000885,0.002860,-0.027728,-0.000253,1.0
2,0.024678,0.009893,0.013634,0.012108,0.012766,1.0
3,0.130142,0.126730,0.127299,0.129483,0.125084,1.0
4,0.000040,-0.000118,0.000135,0.000009,0.000040,1.0
5,-0.092436,-0.010667,0.008220,0.004620,0.007967,1.0
...,...,...,...,...,...,...
596,0.004386,-0.010998,-0.036138,-0.026082,-0.034851,2.0
597,0.176861,0.127837,0.159203,0.113950,0.122719,2.0
598,0.000040,-0.000055,0.000166,0.000135,0.000103,2.0
599,0.039803,0.035824,-0.087225,-0.011709,-0.011741,2.0


## Using XGBoost to classify 

In [98]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [99]:
X = df_main.iloc[:,0:num_features]
Y = df_main.iloc[:,num_features:]
Y.head()

,Label
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0


In [100]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [101]:
model = XGBClassifier()
model.fit(X_train, y_train.values.ravel())

XGBClassifier(objective='multi:softprob')

In [102]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [103]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 84.85%


In [ ]:
df_main.to_csv('Cleaned Data/raw_clean.csv')